# Proyecto 1
### Explorando los Algoritmos de Aprendizaje Supervisado

Estudiantes: 

Carlos Adrián Araya Ramírez---------2018319701

Michael Shakime Richards Sparks----2018170667

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.datasets as datasets
import seaborn as sns


# ANALISIS

In [ ]:
# Load the winequality-red dataset
df = pd.read_csv("datasets/winequality-red.csv")
print(df.describe())

In [ ]:
# Calculate the correlation matrix
corr_matrix = df.corr()
print(corr_matrix)

# VISUALIZACIÓN

In [ ]:
# Visualize the correlation matrix as a heatmap
sns.heatmap(corr_matrix, annot=False, cmap='RdYlBu')

In [ ]:
# Get the list of features
features = df.columns[:-1]

# Define colors for each quality value
colors = {3: 'red', 4: 'orange', 5: 'yellow', 6: 'green', 7: 'blue', 8: 'purple'}

# Create a box plot for each feature
for feature in features:
    # Create a new figure and axis for each feature
    fig, ax = plt.subplots(figsize=(6, 4))
    
    # Loop through each quality value and plot the box
    for quality in sorted(df['quality'].unique()):
        box_color = colors[quality]  # Get the color for this quality value
        data = df[df['quality'] == quality][feature]  # Get the data for this quality value
        ax.boxplot(data, positions=[quality], widths=0.6, showfliers=True, patch_artist=True, boxprops=dict(facecolor=box_color))
    
    # Set the title and axis labels
    plt.title("Box Plot of Wine Quality by " + feature)
    plt.xlabel("Quality")
    plt.ylabel(feature)
    
    # Set the x-axis limits and ticks
    ax.set_xlim(2.5, 8.5)
    ax.set_xticks(range(3, 9))
    ax.set_xticklabels(range(3, 9))
    
    # Remove the grid lines
    ax.grid(False)
    
    # Show the plot
    plt.show()

# Normalización

In [63]:
from sklearn.preprocessing import MinMaxScaler

""" Outliers can skew the mean and standard deviation, resulting in normalization that 
is not representative of the majority of the data points. """

# Get the interquartile range for each column
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Get the lower and upper bounds for each column
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

# Replace the values that are outside the interquartile range with the respective bound
# df_iqr = df.clip(lower=lower, upper=upper, axis=1)

# Remove the values that are outside the interquartile range
df_iqr = df[(df >= lower) & (df <= upper)].dropna()

""" Normalize using MinMaxScaler method. """

# Initialize the MinMaxScaler object
scaler = MinMaxScaler()

# Fit the scaler to the df_iqr dataset
scaler.fit(df_iqr)

# Transform the df_iqr dataset using the scaler
df_normalized = pd.DataFrame(scaler.transform(df_iqr), columns=df_iqr.columns)
print(df_normalized.describe())

       fixed acidity  volatile acidity  citric acid  residual sugar   
count    1179.000000       1179.000000  1179.000000     1179.000000  \
mean        0.425278          0.455442     0.338027        0.410588   
std         0.202538          0.185572     0.245809        0.183738   
min         0.000000          0.000000     0.000000        0.000000   
25%         0.277778          0.305085     0.109589        0.291667   
50%         0.375000          0.451977     0.328767        0.375000   
75%         0.541667          0.576271     0.534247        0.541667   
max         1.000000          1.000000     1.000000        1.000000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density   
count  1179.000000          1179.000000           1179.000000  1179.000000  \
mean      0.481873             0.341960              0.312655     0.488861   
std       0.183553             0.214461              0.225055     0.184419   
min       0.000000             0.000000         